In [3]:
import re
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
dbpia_aut = client['DBPIA']['Author']


In [7]:
def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        # print(input_s)
        # print(c.lower())
        # print(ord(c.lower()))
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>1 else "e"

def check_college(univ0):
    branch_set = ['성균관대학교', '건국대학교', '한양대학교']
    univName = client['PUBLIC']['CollegeName']
    univ1 = re.sub("산학협력단|병원","",univ0)
    univ2 = re.sub("대학교","대학교 ",univ1)
    # univ0 = re.sub("Bursa Yüksek İhtisas |İstanbul |Kahramanmaraş Sütçü İmam |İzmir Katip Celeby University Atatürk |Abant İzzet Baysal |İzmir |İnonu ","",univ0)
    
    try:
        if univ0 == "":
            return univ0
        if isEnglishOrKorean(univ0) == 'e':
            univ0 = univ0.upper()
            univ0 = univ0.replace('.', ',')
            univ = univ0.split(', ')
        else:
            univ = univ2.replace(",", "").split()
            univ = list(set(univ))   
            
        for uni in univ:
            if uni in branch_set:
                if ("ERICA" or "에리카") in univ0:
                    univ[univ.index("한양대학교")] = "한양대학교(ERICA캠퍼스)"
                elif ("글로컬" or "GLOCAL") in univ0:
                    if "건국대학교" in univ0:
                        univ[univ.index("건국대학교")] = "건국대학교 GLOCAL(글로컬)캠퍼스"
                    else :
                        univ[univ.index("성균관대학교")] = "성균관대학교"
                
                elif "자연과학캠퍼스" in univ0:
                    univ[univ.index("성균관대학교")] = "성균관대학교(자연과학캠퍼스)"

        univs = '{"$or": ['
        for u in range(len(univ)):
            if univ[-1] == univ[u]:
                univs += '{"inputName": "' + univ[u] + '"}'
            else:
                univs += '{"inputName": "' + univ[u] + '"}, '
        univs += ']}'

        univ_query = univName.find_one(eval(univs))

        if univ_query is None:
            return univ0
        else:
            return univ_query['originalName']
        
    except SyntaxError as e:
        return univ0


In [8]:
check_college("")

''

In [34]:
count = 0
dbpia_querys = dbpia_aut.find({ 'hasInst' : {'$ne': 'true'}})
for dbpia_query in dbpia_querys[691000:]:
    dbpia_inst = dbpia_query['inst']
    original_inst = check_college(dbpia_inst)
    count += 1
    if count%1000 == 0:
        print(count)
    # print(f"original: {original_inst}, dbpia: {dbpia_inst}")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [12]:
print(ord('i̇'))

TypeError: ord() expected a character, but string of length 2 found